# KFServing Pipeline samples

This notebook assumes your cluster has KFServing >= v0.5.0 installed which supports the v1beta1 API.

### Install the necessary kfp library

In [16]:
import kfp.compiler as compiler
import kfp.dsl as dsl
import kfp
from kfp import components


# Create kfp client
# Note: Add the KubeFlow Pipeline endpoint below if the client is not running on the same cluster.
# Example: kfp.Client('http://192.168.1.27:31380/pipeline')
namespace = 'your_namespace'
client = kfp.Client()
EXPERIMENT_NAME = 'KFServing Experiments'
experiment = client.create_experiment(name=EXPERIMENT_NAME, namespace='andre')

### TensorFlow example

In [17]:
kfserving_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/master/components/kubeflow/kfserving/component.yaml')

@dsl.pipeline(
  name='KFServing pipeline',
  description='A pipeline for KFServing.'
)
def kfservingPipeline(
    action='apply',
    model_name='tensorflow-sample',
    model_uri='gs://kfserving-samples/models/tensorflow/flowers',
    namespace=namespace,
    framework='tensorflow'):

    kfserving = kfserving_op(action = action,
                             model_name=model_name,
                             model_uri=model_uri,
                             namespace=namespace,
                             framework=framework).set_image_pull_policy('Always')

# Compile pipeline
compiler.Compiler().compile(kfservingPipeline, 'tf-flower.tar.gz')

# Execute pipeline
run = client.run_pipeline(experiment.id, 'tf-flower', 'tf-flower.tar.gz')

### Custom model example

In [6]:
kfserving_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/master/components/kubeflow/kfserving/component.yaml')

@dsl.pipeline(
  name='KFServing pipeline',
  description='A pipeline for KFServing.'
)
def kfservingPipeline(
    action='apply',
    model_name='max-image-segmenter',
    namespace=namespace,
    custom_model_spec='{"name": "image-segmenter", "image": "codait/max-image-segmenter:latest", "port": "5000"}'
):

    kfserving = kfserving_op(action=action,
                             model_name=model_name,
                             namespace=namespace,
                             custom_model_spec=custom_model_spec).set_image_pull_policy('Always')

# Compile pipeline
compiler.Compiler().compile(kfservingPipeline, 'custom.tar.gz')

# Execute pipeline
run = client.run_pipeline(experiment.id, 'custom-model', 'custom.tar.gz')